In [1]:
%load_ext lab_black
%load_ext autotime
import pandas as pd
import numpy as np

time: 411 ms (started: 2023-05-28 16:05:35 -07:00)


This notebook is some (potentially time-consuming) data preparation for the  `tfidf-renorm.ipynb` notebook. This notebook downloads the [20 Newsgroups dataset](http://qwone.com/~jason/20Newsgroups/) and [ICLR data](https://github.com/dkobak/iclr-tsne/blob/main/iclr-tsne.ipynb), applies different TF-IDF settings (either L1 or L2 norm, and using linear or sublinear scaling), then SVD and exports the results for later visualization.

In [17]:
from sklearn.decomposition import TruncatedSVD
import sklearn.datasets
import sklearn.feature_extraction.text

from drnb.io import write_pickle
from drnb.log import log


def get_20ng():
    return sklearn.datasets.fetch_20newsgroups_vectorized(subset="all")


def tfidf(data, norm="l1", sublinear=False):
    if not norm:
        norm = None
    return sklearn.feature_extraction.text.TfidfTransformer(
        norm=norm, sublinear_tf=sublinear
    ).fit_transform(data)


def ng20_tfidf(norm="l1", sublinear=False):
    return tfidf(get_20ng().data, norm=norm, sublinear=sublinear)


def tsvd(data, n_components, algorithm="arpack"):
    svd = TruncatedSVD(n_components=n_components, algorithm=algorithm)
    data_pca = svd.fit_transform(data)
    varex = svd.explained_variance_ratio_
    log.info(
        "%d component explains %f %% variance", n_components, np.sum(varex * 100.0)
    )
    return data_pca


def renormalize(data, norm=""):
    if norm == "l2":
        return renormalize_l2(data)
    elif norm == "l1":
        return renormalize_l1(data)
    return data


def renormalize_l1(data):
    return data / np.sum(np.abs(data), axis=1)[:, np.newaxis]


def renormalize_l2(data):
    return data / np.linalg.norm(data, axis=1)[:, np.newaxis]

time: 20.3 ms (started: 2023-05-28 16:11:43 -07:00)


# 20NG

For the SVD settings below, there are a different number of components extracted for each combination of norm and scaling. This was based on re-running the SVD several times with the values in the TF-IDF columns permuted randomly to see if I could detect when the SVD starts fitting "noise". This is very slow so it's not given here.

## L1 normalization

### Sublinear=False

In [4]:
ng20l1l = ng20_tfidf(norm="l1", sublinear=False)

time: 880 ms (started: 2023-05-28 16:05:51 -07:00)


In [19]:
ng20l1l_pca = tsvd(ng20l1l, n_components=2250)
_ = write_pickle(
    ng20l1l_pca,
    "ng20",
    suffix="l1l",
    verbose=False,
    compression="gzip",
    overwrite=True,
)

[16:40:18] INFO     2250 component explains 64.016726 % variance                                   ]8;id=165788;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=538357;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 26min 11s (started: 2023-05-28 16:14:19 -07:00)


### Sublinear=True

In [5]:
ng20l1s = ng20_tfidf(norm="l1", sublinear=True)

time: 873 ms (started: 2023-05-28 16:06:01 -07:00)


In [20]:
ng20l1s_pca = tsvd(ng20l1s, n_components=2750)
_ = write_pickle(
    ng20l1s_pca,
    "ng20",
    suffix="l1s",
    verbose=False,
    compression="gzip",
    overwrite=True,
)

[17:18:48] INFO     2750 component explains 63.327141 % variance                                   ]8;id=2490;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=185868;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 38min 33s (started: 2023-05-28 16:40:30 -07:00)


## L2 normalization

### Sublinear=False

In [6]:
ng20l2l = ng20_tfidf(norm="l2", sublinear=False)

time: 1.19 s (started: 2023-05-28 16:06:38 -07:00)


In [21]:
ng20l2l_pca = tsvd(ng20l2l, n_components=1750)
_ = write_pickle(
    ng20l2l_pca,
    "ng20",
    suffix="l2l",
    verbose=False,
    compression="gzip",
    overwrite=True,
)

[17:34:44] INFO     1750 component explains 54.447172 % variance                                   ]8;id=220625;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=193364;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 15min 50s (started: 2023-05-28 17:19:03 -07:00)


### Sublinear=True

In [23]:
ng20l2s = ng20_tfidf(norm="l2", sublinear=True)

time: 1.38 s (started: 2023-05-28 18:01:25 -07:00)


In [24]:
ng20l2s_pca = tsvd(ng20l2s, n_components=3000)
_ = write_pickle(
    ng20l2s_pca,
    "ng20",
    suffix="l2s",
    verbose=False,
    compression="gzip",
    overwrite=True,
)

[18:54:59] INFO     3000 component explains 60.387367 % variance                                   ]8;id=428901;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=226681;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 53min 48s (started: 2023-05-28 18:01:26 -07:00)


## Pipeline

First, prepare the target labels.

In [25]:
ng20v = get_20ng()

time: 828 ms (started: 2023-05-28 18:55:15 -07:00)


In [26]:
ng20v.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

time: 5.53 ms (started: 2023-05-28 18:55:16 -07:00)


In [27]:
ng20v.target

array([17,  7, 10, ..., 10, 18,  9])

time: 6.02 ms (started: 2023-05-28 18:55:16 -07:00)


Use the `codes_to_categories` function to convert the numeric codes to a category column with the actual newsgroup names:

In [28]:
from drnb.util import codes_to_categories

description = codes_to_categories(
    ng20v.target, ng20v.target_names, col_name="description"
)
description

0        talk.politics.mideast
1                    rec.autos
2             rec.sport.hockey
3             rec.sport.hockey
4                    rec.autos
                 ...          
18841       talk.politics.misc
18842       talk.politics.guns
18843         rec.sport.hockey
18844       talk.politics.misc
18845       rec.sport.baseball
Name: description, Length: 18846, dtype: category
Categories (20, object): ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', ..., 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

time: 43.1 ms (started: 2023-05-28 18:55:16 -07:00)


In [29]:
target = pd.concat([pd.Series(ng20v.target, name="class"), description], axis=1)

time: 6.25 ms (started: 2023-05-28 18:55:16 -07:00)


In [30]:
target

,class,description
0,17,talk.politics.mideast
1,7,rec.autos
2,10,rec.sport.hockey
3,10,rec.sport.hockey
4,7,rec.autos
...,...,...
18841,18,talk.politics.misc
18842,16,talk.politics.guns
18843,10,rec.sport.hockey
18844,18,talk.politics.misc


time: 24.9 ms (started: 2023-05-28 18:55:16 -07:00)


In [31]:
from drnb.io.pipeline import create_default_pipeline

time: 4.86 s (started: 2023-05-28 18:55:16 -07:00)


### L1

#### Sublinear=False

##### Unnormalized

In [32]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1lu",
    data=ng20l1l_pca,
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

[18:55:21] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=398044;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=27356;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 2250)                                                ]8;id=832372;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=964803;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=535507;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=626318;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 2250)                               ]8;id=499037;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=74229;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=410584;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=462598;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l1lu.5.1337.euclidean.csv                    ]8;id=869767;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=34830;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[18:56:19] INFO     Writing numpy format to triplets/ng20l1lu.5.1337.idx.npy                        ]8;id=282303;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=877721;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l1lu.5.1337.euclidean.npy                  ]8;id=328335;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=635368;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1lu                                            ]8;id=117069;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=773516;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 58.1 s (started: 2023-05-28 18:55:21 -07:00)


##### Renormalized

In [33]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1lr",
    data=renormalize(ng20l1l_pca, norm="l1"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=610337;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=117237;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 2250)                                                ]8;id=377320;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=237201;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=612696;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=170438;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 2250)                               ]8;id=26816;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=438066;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=292326;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=113136;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[18:57:14] INFO     Writing csv format to triplets/ng20l1lr.5.1337.euclidean.csv                    ]8;id=21289;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=888008;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/ng20l1lr.5.1337.idx.npy                        ]8;id=135102;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=909640;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l1lr.5.1337.euclidean.npy                  ]8;id=606180;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=353300;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1lr                                            ]8;id=645521;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=65553;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 55.7 s (started: 2023-05-28 18:56:19 -07:00)


#### Sublinear=True

##### Unnormalized

In [34]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1su",
    data=ng20l1s_pca,
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=519341;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=733518;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 2750)                                                ]8;id=142838;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=466424;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=252451;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=279274;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 2750)                               ]8;id=66942;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=46240;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=402078;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=792603;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l1su.5.1337.euclidean.csv                    ]8;id=761168;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=377345;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[18:58:18] INFO     Writing numpy format to triplets/ng20l1su.5.1337.idx.npy                        ]8;id=428273;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=903098;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l1su.5.1337.euclidean.npy                  ]8;id=797733;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=440653;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1su                                            ]8;id=445150;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=678983;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 1min 3s (started: 2023-05-28 18:57:14 -07:00)


##### Renormalized

In [35]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1sr",
    data=renormalize(ng20l1s_pca, norm="l1"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=46477;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=335621;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 2750)                                                ]8;id=238417;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=101823;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=327340;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=648635;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 2750)                               ]8;id=187910;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=18308;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=766561;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=128397;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l1sr.5.1337.euclidean.csv                    ]8;id=51535;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=577007;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[18:59:22] INFO     Writing numpy format to triplets/ng20l1sr.5.1337.idx.npy                        ]8;id=415340;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=486047;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l1sr.5.1337.euclidean.npy                  ]8;id=428560;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=48295;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1sr                                            ]8;id=720698;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=968710;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 1min 4s (started: 2023-05-28 18:58:18 -07:00)


### L2

#### Sublinear=False

##### Unnormalized

In [36]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2lu",
    data=ng20l2l_pca,
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=882938;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=605945;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 1750)                                                ]8;id=525297;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=786747;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=518367;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=357101;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 1750)                               ]8;id=279284;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=400907;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=741191;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=246647;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[19:00:11] INFO     Writing csv format to triplets/ng20l2lu.5.1337.euclidean.csv                    ]8;id=56828;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=762045;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/ng20l2lu.5.1337.idx.npy                        ]8;id=49711;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=140793;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2lu.5.1337.euclidean.npy                  ]8;id=627951;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=776038;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2lu                                            ]8;id=836960;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=204355;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 49.5 s (started: 2023-05-28 18:59:22 -07:00)


##### Renormalized

In [37]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2lr",
    data=renormalize(ng20l2l_pca, norm="l2"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=661784;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=352917;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

[19:00:12] INFO     Initial data shape: (18846, 1750)                                                ]8;id=620329;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=415750;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=277343;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=988193;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 1750)                               ]8;id=183385;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=729899;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=501170;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=310812;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[19:01:02] INFO     Writing csv format to triplets/ng20l2lr.5.1337.euclidean.csv                    ]8;id=471366;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=632049;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/ng20l2lr.5.1337.idx.npy                        ]8;id=362212;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=888556;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2lr.5.1337.euclidean.npy                  ]8;id=976881;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=367509;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2lr                                            ]8;id=159256;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=129099;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 50.7 s (started: 2023-05-28 19:00:11 -07:00)


#### Sublinear=True

##### Unnormalized

In [38]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2su",
    data=ng20l2s_pca,
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=466289;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=854442;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 3000)                                                ]8;id=316688;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=269291;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=579775;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=947327;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 3000)                               ]8;id=150976;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=475877;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=998133;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=444127;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l2su.5.1337.euclidean.csv                    ]8;id=510485;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=575489;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[19:02:10] INFO     Writing numpy format to triplets/ng20l2su.5.1337.idx.npy                        ]8;id=748107;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=971748;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2su.5.1337.euclidean.npy                  ]8;id=837138;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=884788;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2su                                            ]8;id=583412;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=402476;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 1min 7s (started: 2023-05-28 19:01:02 -07:00)


##### Renormalized

In [39]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2sr",
    data=renormalize(ng20l2s_pca, norm="l2"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=421936;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=415818;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 3000)                                                ]8;id=553299;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=24761;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=722338;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=995277;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 3000)                               ]8;id=210196;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=831428;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=603880;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=120466;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[19:03:18] INFO     Writing csv format to triplets/ng20l2sr.5.1337.euclidean.csv                    ]8;id=111259;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=154319;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[19:03:19] INFO     Writing numpy format to triplets/ng20l2sr.5.1337.idx.npy                        ]8;id=173718;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=209130;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2sr.5.1337.euclidean.npy                  ]8;id=499086;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=848058;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2sr                                            ]8;id=572661;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=992249;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 1min 8s (started: 2023-05-28 19:02:10 -07:00)


## 20NG, 500 Components

The downside to using a different number of components for each treatment of the data is that some of the effect in dimensionality reduction may come from the difference in the initial dimensionality. Also, every one of the settings has a much larger number of components extracted than you typically see for most datasets. So here are the data pipelines repeated, but this time only using the first 500 components. That is perhaps still on the large size for what you see a pre-processing for dimensionality reduction, but in the same ballpark as the dimensionality of e.g. BERT embeddings (768D). It also has the disadvantage of being chosen entirely arbitrarily.

### L1

#### Sublinear=False

##### Unnormalized

In [40]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1lu-pca500",
    data=ng20l1l_pca[:, :500],
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

[20:11:29] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=295095;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=375380;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=145632;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=630235;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=38721;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=577132;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=444409;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=126732;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=349757;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=347065;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[20:12:01] INFO     Writing csv format to triplets/ng20l1lu-pca500.5.1337.euclidean.csv             ]8;id=20229;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=247347;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/ng20l1lu-pca500.5.1337.idx.npy                 ]8;id=828237;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=5650;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l1lu-pca500.5.1337.euclidean.npy           ]8;id=144079;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=768607;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1lu-pca500                                     ]8;id=325377;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=754979;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 31.9 s (started: 2023-05-28 20:11:29 -07:00)


##### Renormalized

In [41]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1lr-pca500",
    data=renormalize(ng20l1l_pca[:, :500], norm="l1"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=466242;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=353430;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=504314;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=656537;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=815734;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=527937;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=293690;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=404810;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=940061;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=82907;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l1lr-pca500.5.1337.euclidean.csv             ]8;id=544580;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=113631;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[20:12:33] INFO     Writing numpy format to triplets/ng20l1lr-pca500.5.1337.idx.npy                 ]8;id=535720;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=630230;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l1lr-pca500.5.1337.euclidean.npy           ]8;id=391883;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=913127;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1lr-pca500                                     ]8;id=109981;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=741004;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 31.6 s (started: 2023-05-28 20:12:01 -07:00)


#### Sublinear=True

##### Unnormalized

In [42]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1su-pca500",
    data=ng20l1s_pca[:, :500],
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=995206;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=971641;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=724748;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=800080;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=912731;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=646995;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=100289;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=941187;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=544049;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=611586;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l1su-pca500.5.1337.euclidean.csv             ]8;id=993506;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=342096;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[20:13:05] INFO     Writing numpy format to triplets/ng20l1su-pca500.5.1337.idx.npy                 ]8;id=12584;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=169190;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l1su-pca500.5.1337.euclidean.npy           ]8;id=40815;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=149456;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1su-pca500                                     ]8;id=683065;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=440243;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 31.8 s (started: 2023-05-28 20:12:33 -07:00)


##### Renormalized

In [43]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l1sr-pca500",
    data=renormalize(ng20l1s_pca[:, :500], norm="l1"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=280316;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=995201;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=554426;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=119466;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=158639;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=45633;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=418785;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=596411;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=88850;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=936564;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[20:13:36] INFO     Writing csv format to triplets/ng20l1sr-pca500.5.1337.euclidean.csv             ]8;id=162947;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=36209;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/ng20l1sr-pca500.5.1337.idx.npy                 ]8;id=616604;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=467003;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

[20:13:37] INFO     Writing numpy format to triplets/ng20l1sr-pca500.5.1337.euclidean.npy           ]8;id=930134;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=906624;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l1sr-pca500                                     ]8;id=94257;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=889545;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 31.9 s (started: 2023-05-28 20:13:05 -07:00)


### L2

#### Sublinear=False

##### Unnormalized

In [44]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2lu-pca500",
    data=ng20l2l_pca[:, :500],
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=377193;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=880706;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=705362;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=521638;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=509685;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=945020;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=849329;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=171644;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=798811;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=979617;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l2lu-pca500.5.1337.euclidean.csv             ]8;id=868881;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=361632;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[20:14:09] INFO     Writing numpy format to triplets/ng20l2lu-pca500.5.1337.idx.npy                 ]8;id=68940;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=777644;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2lu-pca500.5.1337.euclidean.npy           ]8;id=366992;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=157285;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2lu-pca500                                     ]8;id=294051;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=941762;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 32.1 s (started: 2023-05-28 20:13:37 -07:00)


##### Renormalized

In [45]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2lr-pca500",
    data=renormalize(ng20l2l_pca[:, :500], norm="l2"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=200242;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=444997;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=540750;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=906540;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=734275;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=42842;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=610596;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=281028;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=176610;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=417374;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l2lr-pca500.5.1337.euclidean.csv             ]8;id=515032;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=928689;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[20:14:41] INFO     Writing numpy format to triplets/ng20l2lr-pca500.5.1337.idx.npy                 ]8;id=467596;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=302979;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2lr-pca500.5.1337.euclidean.npy           ]8;id=814287;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=291795;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2lr-pca500                                     ]8;id=28583;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=16168;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 32.3 s (started: 2023-05-28 20:14:09 -07:00)


#### Sublinear=True

##### Unnormalized

In [46]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2su-pca500",
    data=ng20l2s_pca[:, :500],
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=656650;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=668395;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=472526;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=449458;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=574364;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=878727;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=815268;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=975726;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=219443;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=614076;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/ng20l2su-pca500.5.1337.euclidean.csv             ]8;id=558968;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=485333;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[20:15:14] INFO     Writing numpy format to triplets/ng20l2su-pca500.5.1337.idx.npy                 ]8;id=655745;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=575542;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2su-pca500.5.1337.euclidean.npy           ]8;id=806407;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=539796;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2su-pca500                                     ]8;id=258466;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=608959;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 33.1 s (started: 2023-05-28 20:14:41 -07:00)


##### Renormalized

In [47]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "ng20l2sr-pca500",
    data=renormalize(ng20l2s_pca[:, :500], norm="l2"),
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=433996;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=639009;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (18846, 500)                                                 ]8;id=195034;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=948641;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=883710;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=613507;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (18846, 500)                                ]8;id=94771;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=290878;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=807463;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=660160;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[20:15:46] INFO     Writing csv format to triplets/ng20l2sr-pca500.5.1337.euclidean.csv             ]8;id=527987;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=988406;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/ng20l2sr-pca500.5.1337.idx.npy                 ]8;id=340468;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=877548;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/ng20l2sr-pca500.5.1337.euclidean.npy           ]8;id=421798;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=675575;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for ng20l2sr-pca500                                     ]8;id=632803;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=380103;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 32.3 s (started: 2023-05-28 20:15:14 -07:00)


# iclr

See (and run) the `data-pipeline/iclr.pynb` notebook for where the text data comes from ([iclr abstract data](https://github.com/dkobak/iclr-tsne/blob/main/iclr-tsne.ipynb) originally analyzed by Dmitry Kobak).

In [50]:
from drnb.io import read_pickle

time: 1.58 ms (started: 2023-05-28 22:19:35 -07:00)


In [51]:
iclr_text = read_pickle("iclr", suffix="text")

time: 842 ms (started: 2023-05-28 22:19:36 -07:00)


Some of this data has been processed as part of the `iclr.pynb` notebook which uses Dmitry's setting of L2 normalization, sublinear scaling, and extracting 100 components. Below, I will try the same combination of norm/scaling as used with 20NG, but with 200 components to avoid being too duplicative of the previous datasets. 100-200 components is a reasonable choice for all the norm/scaling settings based on comparing the amount of variance extracted at different components vs re-running SVD multiple times with the values in each column randomly permuted, so I've chosen the higher end of the estimate for these runs.

In [53]:
def tfidfv(data, norm="l1", sublinear=False):
    if not norm:
        norm = None
    return sklearn.feature_extraction.text.TfidfVectorizer(
        norm=norm, sublinear_tf=sublinear
    ).fit_transform(data)

time: 2.51 ms (started: 2023-05-28 22:21:26 -07:00)


In [60]:
from drnb.io.dataset import read_target, read_palette

iclr_target = read_target("iclr")
iclr_palette = read_palette("iclr")

time: 130 ms (started: 2023-05-28 22:25:11 -07:00)


## L1 normalization

### Sublinear=False

In [54]:
iclrl1l = tfidfv(iclr_text, norm="l1", sublinear=False)
iclrl1l_pca = tsvd(iclrl1l, n_components=200)

[22:21:47] INFO     200 component explains 24.559193 % variance                                    ]8;id=263984;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=177325;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 14.2 s (started: 2023-05-28 22:21:32 -07:00)


#### Unnormalized

In [62]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl1lu-pca200",
    data=iclrl1l_pca,
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

[22:27:57] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=366128;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=688398;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=274895;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=575953;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=930153;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=338880;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=688579;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=161399;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=366932;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=153351;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/iclrl1lu-pca200.5.1337.euclidean.csv             ]8;id=749318;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=599158;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/iclrl1lu-pca200.5.1337.idx.npy                 ]8;id=667372;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=382670;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

[22:28:22] INFO     Writing numpy format to triplets/iclrl1lu-pca200.5.1337.euclidean.npy           ]8;id=6823;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=500082;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl1lu-pca200                                     ]8;id=279451;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=261705;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 24.1 s (started: 2023-05-28 22:27:57 -07:00)


#### Renormalized

In [63]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl1lr-pca200",
    data=renormalize(iclrl1l_pca, norm="l1"),
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=765301;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=245092;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=222344;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=780975;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=856575;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=672369;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=632931;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=969733;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=593331;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=639493;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[22:28:46] INFO     Writing csv format to triplets/iclrl1lr-pca200.5.1337.euclidean.csv             ]8;id=230174;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=526438;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/iclrl1lr-pca200.5.1337.idx.npy                 ]8;id=694171;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=417236;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/iclrl1lr-pca200.5.1337.euclidean.npy           ]8;id=411264;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=895538;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl1lr-pca200                                     ]8;id=870355;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=466560;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 24.8 s (started: 2023-05-28 22:28:22 -07:00)


### Sublinear=True

In [55]:
iclrl1s = tfidfv(iclr_text, norm="l1", sublinear=True)
iclrl1s_pca = tsvd(iclrl1s, n_components=200)

[22:22:16] INFO     200 component explains 19.940522 % variance                                    ]8;id=574582;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=404552;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 16.1 s (started: 2023-05-28 22:22:00 -07:00)


#### Unnormalized

In [64]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl1su-pca200",
    data=iclrl1s_pca,
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

[22:29:39] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=606629;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=708622;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=703716;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=58484;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=503907;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=633549;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=20478;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=934931;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=413647;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=712601;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/iclrl1su-pca200.5.1337.euclidean.csv             ]8;id=997556;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=854827;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

[22:30:03] INFO     Writing numpy format to triplets/iclrl1su-pca200.5.1337.idx.npy                 ]8;id=416917;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=186393;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/iclrl1su-pca200.5.1337.euclidean.npy           ]8;id=598145;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=851261;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl1su-pca200                                     ]8;id=776188;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=622763;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 24 s (started: 2023-05-28 22:29:39 -07:00)


#### Renormalized

In [70]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl1sr-pca200",
    data=renormalize(iclrl1s_pca, norm="l1"),
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

[22:34:21] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=936506;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=497992;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=46208;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=805263;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=84105;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=439249;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=564099;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=947354;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=314701;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=421825;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[22:34:44] INFO     Writing csv format to triplets/iclrl1sr-pca200.5.1337.euclidean.csv             ]8;id=606719;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=549334;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/iclrl1sr-pca200.5.1337.idx.npy                 ]8;id=267039;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=30436;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/iclrl1sr-pca200.5.1337.euclidean.npy           ]8;id=980398;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=492341;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl1sr-pca200                                     ]8;id=560675;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=64897;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 23.7 s (started: 2023-05-28 22:34:21 -07:00)


## L2 normalization

### Sublinear=False

In [57]:
iclrl2l = tfidfv(iclr_text, norm="l2", sublinear=False)
iclrl2l_pca = tsvd(iclrl2l, n_components=200)

[22:22:52] INFO     200 component explains 24.837214 % variance                                    ]8;id=764965;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=305951;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 13.2 s (started: 2023-05-28 22:22:38 -07:00)


#### Unnormalized

In [66]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl2lu-pca200",
    data=iclrl2l_pca,
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

[22:31:33] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=193180;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=163415;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=322350;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=447002;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=191711;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=240759;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=208145;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=295304;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=713218;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=176243;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/iclrl2lu-pca200.5.1337.euclidean.csv             ]8;id=376453;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=736836;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/iclrl2lu-pca200.5.1337.idx.npy                 ]8;id=357554;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=735017;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

[22:31:57] INFO     Writing numpy format to triplets/iclrl2lu-pca200.5.1337.euclidean.npy           ]8;id=879758;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=784037;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl2lu-pca200                                     ]8;id=305002;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=499066;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 23.6 s (started: 2023-05-28 22:31:33 -07:00)


#### Renormalized

In [67]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl2lr-pca200",
    data=renormalize(iclrl2l_pca, norm="l2"),
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

[22:31:59] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=449599;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=529317;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=175241;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=924076;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=72943;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=811134;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=203386;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=134465;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=477318;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=116283;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[22:32:22] INFO     Writing csv format to triplets/iclrl2lr-pca200.5.1337.euclidean.csv             ]8;id=884791;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=406462;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/iclrl2lr-pca200.5.1337.idx.npy                 ]8;id=151179;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=852410;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/iclrl2lr-pca200.5.1337.euclidean.npy           ]8;id=343904;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=824823;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl2lr-pca200                                     ]8;id=544642;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=811347;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 23.5 s (started: 2023-05-28 22:31:59 -07:00)


### Sublinear=True

In [58]:
iclrl2s = tfidfv(iclr_text, norm="l2", sublinear=True)
iclrl2s_pca = tsvd(iclrl2s, n_components=200)

[22:23:07] INFO     200 component explains 19.825772 % variance                                    ]8;id=561530;file:///tmp/ipykernel_28960/2797199864.py\2797199864.py]8;;\:]8;id=801425;file:///tmp/ipykernel_28960/2797199864.py#28\28]8;;\

time: 15.8 s (started: 2023-05-28 22:22:52 -07:00)


#### Unnormalized

In [68]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl2su-pca200",
    data=iclrl2s_pca,
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

[22:32:57] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=188757;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=194407;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=22472;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=378931;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=912793;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=983179;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=406254;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=918815;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=569659;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=209177;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[22:33:21] INFO     Writing csv format to triplets/iclrl2su-pca200.5.1337.euclidean.csv             ]8;id=94792;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=220758;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/iclrl2su-pca200.5.1337.idx.npy                 ]8;id=974184;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=956823;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/iclrl2su-pca200.5.1337.euclidean.npy           ]8;id=415562;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=806920;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl2su-pca200                                     ]8;id=774381;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=983903;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 24.3 s (started: 2023-05-28 22:32:57 -07:00)


#### Renormalized

In [69]:
_ = create_default_pipeline(
    check_for_duplicates=True,
    metric=["euclidean"],
).run(
    "iclrl2sr-pca200",
    data=renormalize(iclrl2s_pca, norm="l2"),
    target=iclr_target,
    target_palette=iclr_palette,
    url="https://github.com/dkobak/iclr-tsne",
    verbose=True,
)

[22:33:48] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=982697;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=971967;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#566\566]8;;\

           INFO     Initial data shape: (16554, 200)                                                 ]8;id=371179;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=838701;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=276155;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=63637;file:///home/james/dev/drnb/src/drnb/preprocess.py#67\67]8;;\

           INFO     Data shape after filtering columns: (16554, 200)                                ]8;id=961071;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=532318;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=890499;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=566224;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

[22:34:11] INFO     Writing csv format to triplets/iclrl2sr-pca200.5.1337.euclidean.csv             ]8;id=507911;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=143067;file:///home/james/dev/drnb/src/drnb/io/__init__.py#263\263]8;;\

           INFO     Writing numpy format to triplets/iclrl2sr-pca200.5.1337.idx.npy                 ]8;id=951672;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=945943;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing numpy format to triplets/iclrl2sr-pca200.5.1337.euclidean.npy           ]8;id=319268;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=141655;file:///home/james/dev/drnb/src/drnb/io/__init__.py#286\286]8;;\

           INFO     Writing pipeline result for iclrl2sr-pca200                                     ]8;id=404980;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=849386;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 23.7 s (started: 2023-05-28 22:33:48 -07:00)
